In [1]:
# Introduce
# Jest to modul wyliczenia danych wejsciowych dla modulu predykcji wynikow rozgrywej sportowych

# Dane pobrane zosataly ze strony http://www.football-data.co.uk, 
# pelny opis danych mozna zobaczyc na stronie http://www.football-data.co.uk/notes.txt
# Pobrane dane mam podzielone na poszczegolne sezony

# Do wyliczenia moich danych wejsciowych wykorzystalem nastepujace dane 
# AwayTeam - nazwa druzyny goscia
# HomeTeam - nazwa druzyny gospodarzy
# Date - data rozegrania spotkania
# FTAG - liczba strzelnych bramek podczas meczu przez druzyne gosci
# FTHG - liczba strzelnych bramek podczas meczu przez druzyne gospodarzy
# FTR - wynik koncowy spotkania (H - wygrana gospodarzy, D - remis, A - wygrana gosci)

# Na podstawie tych danych wyliczylem nastepujace dane:
# MW - kolejka podczas ktorej bylo rozgrywane spotkanie

# Ponizsze dane wykonalem dla ostatnich 5,10,20,30,i 38 meczow w danym sezonie, jezeli dana zmienna dotyczy 30 meczow
# to ma wtedy koncowke zawierajaca liczbe 30 np HGD30 , 38 to maksymalna liczba poniewaz tyle meczy 
# zostaje rozegranych podczas jednego sezony 
# HGS10 - gole strzelone przez gospodarzy w ostatnich 10 meczach
# HGL10 - gole stracone przez gospodarzy w ostatnich 10 meczach
# HGD10 - wyliaczne ze wedlug wzoru HGD10=HGS10-HGL10  
# AGS10 - gole strzelone przez gosci w ostatnich 10 meczach      
# AGL10 - gole stracone przez gosci w ostatnich 10 meczach
# AGD10 - wyliaczne ze wedlug wzoru AGD10=AGS10-AGL10 
# GD10 - roznica bilanu bramke obu druzyn wyliczna wedlug wzoru : GD10 = HGD10-AGD10 

# HP10 - suma punktow zdobytych przez gospodarzy w oostatnich 10 meczach 
# AP10 - suma punktow zdobytych przez gosci w oostatnich 10 meczach                
# DP10 - roznica punktow zdobytch przez obie druzyny wyliczana wedlug wzoru DP10=HP10-AP10 

#Ponizesz dane nie byly juz wyliczane dla interwalow
# HF5 - forma gospodarzy w ostatnich 5 meczach np. (WLDWW)
# AF5 - forma gosci w ostatnich 5 meczach
# HLF5 - punkty gospodarzy wedlug formy gospodarzy wyliczane ze zmiennej HF5 (patrz funkcja 'calc_points_based_on_form')
# ALF5 - punkty gosci wedlug formy gosci wyliczane ze zmiennej AF5 (patrz funkcja 'calc_points_based_on_form')
# LFD5 - roznica punktw formy wyliczna wedlug wzoru LFD5=HLF5-ALF5

# LSTPH - miejsce w tabeli druzyny gospodarzy w poprzednim sezonie      
# LSTPA - miejsce w tabeli druzyny gosci w poprzednim sezonie      
# LSTPD - roznica pomiedzy pozycjami w tabeli w ostanim sezonie pomiedzy druzynami, obliczana wedlug wzoru LSTPD=LSTPH-LSTPA  

# Rozbijam zmienna data
# Day        
# Month            
# Year           
# DaysFromToday - liczba ktore minely od dnia dzisiejszego  

# Do tej pory dane mialem rozbite na poszczegolne sezony , teraz lacze wszystie sezony w jeden zbior
# i dla interwalow 5,10,20,30,i 38 wyliczam nastepujace dane  
# DPB10 - roznica punktow zdobytych przez gospodarzy i gosci w ostatnich 10 meczach pomiedzy tymi druzynami

# Do moich danych wejsciowych dolaczylem dane AwayTeam,HomeTeam,FTR pochodzace z pobranego zbioru danych

# Do predykcji wyniku spotkania potrzebne sa tylko nazwy druzyn gospodarzy i gosci , 
# Cala reszta danych wyliczana jest na podstawie danych historycznych policzonych przez ten modul

In [2]:
import pandas as pd
import sys
import datetime
import numpy as np 
import scipy as sc
import scipy.stats

In [5]:
c1 = pd.concat(map(pd.read_csv, ['data/E0.csv']),ignore_index=True)
c2 = pd.concat(map(pd.read_csv, ['data/E1.csv']),ignore_index=True)
c3 = pd.concat(map(pd.read_csv, ['data/E2.csv']),ignore_index=True)
c4 = pd.concat(map(pd.read_csv, ['data/E3.csv']),ignore_index=True)
c5 = pd.concat(map(pd.read_csv, ['data/E4.csv']),ignore_index=True)
c6 = pd.concat(map(pd.read_csv, ['data/E5.csv']),ignore_index=True)
c7 = pd.concat(map(pd.read_csv, ['data/E6.csv']),ignore_index=True)

In [6]:
def add_game_week(df):
    MatchWeek = []
    for i, row in df.iterrows():
         df.loc[i,'MW']  = int((i+10) / 10)    
    return df

c1 = add_game_week(c1)
c2 = add_game_week(c2)
c3 = add_game_week(c3)
c4 = add_game_week(c4)
c5 = add_game_week(c5)
c6 = add_game_week(c6)
c7 = add_game_week(c7)

In [8]:
c1 = c1.iloc[::-1]
c1 = c1.reset_index(drop=True)
c2 = c2.iloc[::-1]
c2 = c2.reset_index(drop=True)
c3 = c3.iloc[::-1]
c3 = c3.reset_index(drop=True)
c4 = c4.iloc[::-1]
c4 = c4.reset_index(drop=True)
c5 = c5.iloc[::-1]
c5 = c5.reset_index(drop=True)
c6 = c6.iloc[::-1]
c6 = c6.reset_index(drop=True)
c7 = c7.iloc[::-1]
c7 = c7.reset_index(drop=True)

In [9]:
c1 = c1[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c2 = c2[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c3 = c3[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c4 = c4[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c5 = c5[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c6 = c6[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]
c7 = c7[['AwayTeam','Date','FTAG','FTHG','FTR','HomeTeam','MW']]

In [11]:
c1 = c1[c1.HomeTeam.notnull()]
print(c1.isnull().values.any())

c2 = c2[c2.HomeTeam.notnull()]
print(c2.isnull().values.any())

c3 = c3[c3.HomeTeam.notnull()]
print(c3.isnull().values.any())

c4 = c4[c4.HomeTeam.notnull()]
print(c4.isnull().values.any())

c5 = c5[c5.HomeTeam.notnull()]
print(c5.isnull().values.any())

c6 = c6[c6.HomeTeam.notnull()]
print(c6.isnull().values.any())

c7 = c7[c7.HomeTeam.notnull()]
print(c7.isnull().values.any())

False
False
False
False
False
False
False


In [12]:
def count_goal_diff(number_of_last_matches,team_side,column,df,counting_matches_from,column_score = None, column_lost = None):
    for i, row in df.iterrows():
        team = df.loc[i,team_side]
        goal_diff = 0
        goal_score = 0
        goal_lost =0
        iterator = 0
    
        for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                goal_diff += df.loc[(i+j),'FTHG'] - df.loc[(i+j),'FTAG']
                goal_score += df.loc[(i+j),'FTHG']
                goal_lost += df.loc[(i+j),'FTAG']
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                goal_diff += df.loc[(i+j),'FTAG'] - df.loc[(i+j),'FTHG']
                goal_score += df.loc[(i+j),'FTAG']
                goal_lost += df.loc[(i+j),'FTHG']
                iterator += 1
            if iterator == number_of_last_matches:
                break
        df.loc[i,column] = goal_diff
        if column_score != None:
            df.loc[i,column_score] = goal_score 
        if column_lost != None:
            df.loc[i,column_lost] = goal_lost 
    
def count_goals_diff_for_both_teams(df,number_of_last_matches):
    matches = str(number_of_last_matches)
    
    count_goal_diff(number_of_last_matches,'HomeTeam','HGD'+matches,df,1,'HGS'+matches,'HGL'+matches)
    count_goal_diff(last_matches,'AwayTeam','AGD'+matches,df,1,'AGS'+matches,'AGL'+matches)
    
    for i, row in df.iterrows():
        df.loc[i,'GD'+matches] = df.loc[i,'HGD'+matches] - df.loc[i,'AGD'+matches]        
    return df

#zmienna potrzebna do wyliczenia pozycji w ostatnim sezonie
count_goal_diff(38,'HomeTeam','HGDT',c2,0)
count_goal_diff(38,'AwayTeam','AGDT',c2,0)
count_goal_diff(38,'HomeTeam','HGDT',c3,0)
count_goal_diff(38,'AwayTeam','AGDT',c3,0)
count_goal_diff(38,'HomeTeam','HGDT',c4,0)
count_goal_diff(38,'AwayTeam','AGDT',c4,0)
count_goal_diff(38,'HomeTeam','HGDT',c5,0)
count_goal_diff(38,'AwayTeam','AGDT',c5,0)
count_goal_diff(38,'HomeTeam','HGDT',c6,0)
count_goal_diff(38,'AwayTeam','AGDT',c6,0)
count_goal_diff(38,'HomeTeam','HGDT',c7,0)
count_goal_diff(38,'AwayTeam','AGDT',c7,0)

#roznica strzelonych bramek w n ostatnich meczach historycznych
number_of_last_matches = [5,10,20,30,38]

for last_matches in number_of_last_matches:
    c1= count_goals_diff_for_both_teams(c1,last_matches)
    c2= count_goals_diff_for_both_teams(c2,last_matches)
    c3= count_goals_diff_for_both_teams(c3,last_matches)
    c4= count_goals_diff_for_both_teams(c4,last_matches)
    c5= count_goals_diff_for_both_teams(c5,last_matches)
    c6= count_goals_diff_for_both_teams(c6,last_matches)
    c7= count_goals_diff_for_both_teams(c7,last_matches)

In [13]:
def get_points(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_points_reverse(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def calc_team_point(number_of_last_matches,team,i,df,counting_matches_from):
    points = 0
    iterator = 0
    for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                points += get_points(df.loc[(i+j),'FTR'])
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                points += get_points_reverse(df.loc[(i+j),'FTR'])
                iterator += 1
            if iterator == number_of_last_matches:
                break 
    return points

def calc_points(df,number_of_last_matches):
    matchs = str(number_of_last_matches)
    
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df.loc[i,'HP'+matchs] = calc_team_point(number_of_last_matches,home_team,i,df,1) 
        df.loc[i,'AP'+matchs] = calc_team_point(number_of_last_matches,away_team,i,df,1)
        
        #zmienna potrzebna do wyliczenia pozycji w ostatnim sezonie
        df.loc[i,'HPT'] = calc_team_point(38,home_team,i,df,0) 
        df.loc[i,'APT'] = calc_team_point(38,away_team,i,df,0)
    
    for i, row in df.iterrows():
        df.loc[i,'DP'+matchs] = df.loc[i,'HP'+matchs] - df.loc[i,'AP'+matchs]   
    return df


number_of_last_matches = [5,10,20,30,38]

for last_matches in number_of_last_matches:
    c1= calc_points(c1,last_matches)
    c2= calc_points(c2,last_matches)
    c3= calc_points(c3,last_matches)
    c4= calc_points(c4,last_matches)
    c5= calc_points(c5,last_matches)
    c6= calc_points(c6,last_matches)
    c7= calc_points(c6,last_matches)

In [14]:
def get_points(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_points_reverse(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    
def get_match_result(points):
    if points == 3:
        return 'W'
    elif points == 1:
        return 'D'
    elif points == 0:
        return 'L'
    else:
        return 'N'

def calc_team_point(number_of_last_matches,team,i,df,counting_matches_from):
    points = ""
    iterator = 0
    for j in range(counting_matches_from,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == team:
                points += get_match_result(get_points(df.loc[(i+j),'FTR']))
                iterator += 1
            elif df.loc[(i+j),'AwayTeam'] == team:
                points += get_match_result(get_points_reverse(df.loc[(i+j),'FTR']))
                iterator += 1
            if iterator == number_of_last_matches:
                break 
    return points

def calc_points(df,number_of_last_matches):
    matches = str(number_of_last_matches)
    
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df.loc[i,'HF'+matches] = calc_team_point(number_of_last_matches,home_team,i,df,1) 
        df.loc[i,'AF'+matches] = calc_team_point(number_of_last_matches,away_team,i,df,1)
    return df

c1 = calc_points(c1,5)
c2 = calc_points(c2,5)
c3 = calc_points(c3,5)
c4 = calc_points(c4,5)
c5 = calc_points(c5,5)
c6 = calc_points(c6,5)
c7 = calc_points(c7,5)

In [15]:
def calc_points_based_on_form(string):
    if string == 'WWWWW':
        return 5
    elif string[-4:] == 'WWWWW':
        return 4
    elif string[-3:] == 'WWW':
        return 3
    elif string[-2:] == 'WW':
        return 2
    elif string == 'LLLLL':
        return -5
    elif string[-4:] == 'LLLL':
        return -4
    elif string[-3:] == 'LLL':
        return -3
    elif string[-2:] == 'LL':
        return -2
    else:
        return 0
    
def check_last_teams_form(df):
    for i, row in df.iterrows():
        home_team_form = df.loc[(i),'HF5']
        away_team_form = df.loc[(i),'AF5']
        df.loc[i,'HLF5'] = calc_points_based_on_form(home_team_form) 
        df.loc[i,'ALF5'] = calc_points_based_on_form(away_team_form)
    
    for i, row in df.iterrows():
        df.loc[i,'LFD5'] = df.loc[i,'HLF5'] - df.loc[i,'ALF5']
    
    return df

c1 = check_last_teams_form(c1)
c2 = check_last_teams_form(c2)
c3 = check_last_teams_form(c3)
c4 = check_last_teams_form(c4)
c5 = check_last_teams_form(c5)
c6 = check_last_teams_form(c6)
c7 = check_last_teams_form(c7)

In [16]:
def get_team(df):
    return pd.DataFrame({'AwayTeam':df.AwayTeam.unique()})

def calc_points_table(df):
    dictionary = {}
    teams = get_team(df)
    for i, row in teams.iterrows():
        points = 0
        goal_diff = 0 
        team = teams.loc[(i),'AwayTeam']
        
        team_matches = df.loc[((df['AwayTeam'] == team) | (df['HomeTeam'] == team))]
        
        team_matches = team_matches.reset_index(drop=True)
        
        if team_matches.loc[(0),'HomeTeam'] == team:
            points = team_matches.loc[(0),'HPT']
            goal_diff = team_matches.loc[(0),'HGDT']
        elif team_matches.loc[(0),'AwayTeam'] == team:
            points = team_matches.loc[(0),'APT']
            goal_diff = team_matches.loc[(0),'AGDT']
            
        dictionary[team] = (points,goal_diff)
        
    return {key: rank for rank, key in enumerate(sorted(dictionary, key=dictionary.get, reverse=True), 1)}

def assing_position_in_table_to_team_in_last_season(table,df):
    positions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19] 
    df['LSTPH'] = df['HomeTeam']
    df['LSTPA'] = df['AwayTeam']
    
    for team in get_team(df)['AwayTeam']:
        if not team in table:
            df.LSTPH = df.LSTPH.replace([team], [20])
            df.LSTPA = df.LSTPA.replace([team], [20])
            
    df.LSTPH = df.LSTPH.astype(str)
    df.LSTPA = df.LSTPA.astype(str)
    
    for key, value in table.items():
        df.LSTPH = df.LSTPH.replace([key], [value])
        df.LSTPA = df.LSTPA.replace([key], [value])
        
    df.LSTPH = df.LSTPH.astype(int)
    df.LSTPA = df.LSTPA.astype(int)
        
    for i, row in df.iterrows():
        df.loc[i,'LSTPD'] = df.loc[i,'LSTPH'] - df.loc[i,'LSTPA']
    
    return df

table_c2 = calc_points_table(c2)
table_c3 = calc_points_table(c3)
table_c4 = calc_points_table(c4)
table_c5 = calc_points_table(c5)
table_c6 = calc_points_table(c6)
table_c7 = calc_points_table(c7)

c1 = assing_position_in_table_to_team_in_last_season(table_c2,c1)
c2 = assing_position_in_table_to_team_in_last_season(table_c3,c2)
c3 = assing_position_in_table_to_team_in_last_season(table_c4,c3)
c4 = assing_position_in_table_to_team_in_last_season(table_c5,c4)
c5 = assing_position_in_table_to_team_in_last_season(table_c6,c5)
c6 = assing_position_in_table_to_team_in_last_season(table_c7,c6)

In [17]:
def parse_day(df):
    newest_date = datetime.datetime.strptime(str(max(df['Date'])), '%Y-%m-%d').date()
    for i, row in df.iterrows():
        df.loc[i,'Day']  = df.at[i,'Date'].day 
        df.loc[i,'Month']  = df.at[i,'Date'].month
        df.loc[i,'Year']  = df.at[i,'Date'].year
        delta = newest_date - df.at[i,'Date']
        df.loc[i,'DaysFromToday']  = delta.days
        
def convert_float_to_int(df):
    columns_to_convert = ['FTAG','FTHG','Day','Month','Year','DaysFromToday']
    for c in columns_to_convert:
        df[c]= df[c].astype(int)
        
def convert_date(df):
    df['Date'] =pd.to_datetime(df['Date'],format='%d/%m/%y')
    df['Date'] = df['Date'].dt.date.apply(lambda x: datetime.date(x.year,x.month,x.day))
    df= df.sort_values(by=['Date'],ascending=False)
    parse_day(df)
    convert_float_to_int(df) 
    return df

def convert_another_date_format(df):
    df['Date'] =pd.to_datetime(df['Date'],format='%Y-%m-%d')
    df['Date'] = df['Date'].dt.date.apply(lambda x: datetime.date(x.year,x.month,x.day))
    df= df.sort_values(by=['Date'],ascending=False)
    parse_day(df)
    convert_float_to_int(df) 
    return df


c1= convert_date(c1)
c2= convert_date(c2)
c3= convert_date(c3)
c4= convert_date(c4)
c5= convert_date(c5)
c6= convert_date(c6)
c7= convert_another_date_format(c7)

In [19]:
filepaths = [c1,c2,c3,c4,c5,c6]
df = pd.concat(filepaths,ignore_index=True)

In [20]:
def get_points_between(result):
    if result == 'H':
        return 3
    elif result == 'D':
        return 0
    else:
        return -3
    
def get_points_reverse_between(result):
    if result == 'A':
        return 3
    elif result == 'D':
        return 0
    else:
        return -3

def calc_point(number_of_last_matches,home_team,away_team,i,df):
    points=0
    iterator = 0
    for j in range(1,len(df)-i):
            if df.loc[(i+j),'HomeTeam'] == home_team and df.loc[(i+j),'AwayTeam'] == away_team:
                points += get_points_between(df.loc[(i+j),'FTR'])
                iterator += 1
            elif df.loc[(i+j),'HomeTeam'] == away_team and df.loc[(i+j),'AwayTeam'] == home_team:
                points += get_points_reverse_between(df.loc[(i+j),'FTR'])
                iterator += 1
            if iterator == number_of_last_matches:
                break 
  
    matches = str(number_of_last_matches)
    df.loc[i,'DPB'+matches] = points         
    return df

def calc_points_between_opponents(df,number_of_last_matches):
    for i, row in df.iterrows():
        home_team = df.loc[(i),'HomeTeam']
        away_team = df.loc[(i),'AwayTeam']
        df = calc_point(number_of_last_matches,home_team,away_team,i,df) 
    
    return df

number_of_last_matches = [5,10,20,30,40]

for last_matches in number_of_last_matches:
    df = calc_points_between_opponents(df,last_matches)

In [21]:
columns = ['FTHG','FTAG','AGDT','APT','HGDT','HPT','Date']
df.drop(columns, inplace=True, axis=1)

In [22]:
df.to_csv('football_data.csv', sep=',',index=False)

In [23]:
df.head()

,AF5,AGD10,AGD20,AGD30,AGD38,AGD5,AGL10,AGL20,AGL30,AGL38,...,LSTPD,LSTPH,MW,Month,Year,DPB5,DPB10,DPB20,DPB30,DPB40
0,DWWDD,2.0,-4.0,-4.0,-12.0,3.0,9.0,26.0,43.0,55.0,...,4.0,11,38.0,5,2018,-3.0,-18.0,-21.0,-21.0,-21.0
1,LWDDL,-2.0,-7.0,-12.0,-16.0,-2.0,14.0,27.0,41.0,50.0,...,-16.0,4,38.0,5,2018,3.0,3.0,3.0,3.0,3.0
2,WLLLD,-7.0,-12.0,-10.0,-17.0,-5.0,19.0,40.0,49.0,60.0,...,7.0,16,38.0,5,2018,3.0,3.0,3.0,3.0,3.0
3,WWDWD,-4.0,-13.0,-21.0,-23.0,3.0,14.0,32.0,46.0,54.0,...,4.0,14,38.0,5,2018,0.0,-3.0,-3.0,-3.0,-3.0
4,LWLWL,7.0,12.0,19.0,22.0,4.0,15.0,31.0,43.0,51.0,...,15.0,20,38.0,5,2018,-3.0,-3.0,-3.0,-3.0,-3.0
